In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
import gensim.downloader

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
class Textpreprocessing:
  def __init__(self,text):
    self.text=text

  def words(self,str1):
    tokens=str1.split()[:4]
    str2=''

    for i in range(len(tokens)):
      if i==0:
        str2+=tokens[i]
      else:
        str2=str2+" "+tokens[i]

    return str2



  def stopword_remove(self,str1):
    str1=str1.lower()
    tokens=str1.split()
    str2=""
    stop_words=stopwords.words('english')
    for word in tokens:
      if not word in stop_words:
        str2+=word+' '
    return str2

  def url_remove(self,str1):
    str1=re.sub(r'http\S+', '', str1)
    str1=re.sub(r'www\S+', '', str1)
    return str1

  def clean_punctuation(self,str1):
    str1=re.sub(r'[^\w\s]','',str1)
    return str1

  def cleaningdigits(self,str1):
    str1=re.sub(r'[\d+]','',str1)
    return str1

  def lemmatization(self,str1):
    lemma=WordNetLemmatizer()
    str2=''
    tokens=str1.split()
    for word in tokens:
      store=lemma.lemmatize(word)
      str2+=store+' '
    return str2

  def preprocess(self):
    self.text=self.text.apply(self.words)
    self.text=self.text.apply(self.stopword_remove)
    self.text=self.text.apply(self.url_remove)
    self.text=self.text.apply(self.clean_punctuation)
    self.text=self.text.apply(self.cleaningdigits)
    self.text=self.text.apply(self.lemmatization)

    return self.text

In [ ]:
def convert_to_number(label):
  if label=='spam':
    return 1
  return 0

In [ ]:
url='https://drive.google.com/file/d/1xEyhcHjrjEo62k84kKu6mI9vttGtc-jV/view?usp=share_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
df=df.drop_duplicates()
df=df[df['Message'].notnull()]
df=df[df['Category'].notnull()]

print(df.head)

NameError: ignored

In [ ]:
text_preprocess=Textpreprocessing(df['Message'])
df['Message']=text_preprocess.preprocess()
document_size=len(df)



NameError: ignored

In [ ]:
df['Numeric_label']=df['Category'].apply(convert_to_number)


In [ ]:
#change glove overall
new_model=gensim.downloader.load('glove-twitter-25')
new_model.most_similar("bad")
wv_size=len(new_model['bad'])

In [ ]:
def create_word2vec_for_setence(text,max_word):

  results  = np.zeros(shape = (max_word,wv_size))
  #for i, documents in enumerate(text):
  for j, considered_word in list(enumerate(text.split())):
    try:
      results[j, :] = new_model[considered_word]
    except:
      results[j,:]=np.zeros((wv_size))
  return torch.tensor(results,dtype=torch.float32)

In [ ]:
class MyDataset(Dataset):
  def __init__(self,df):
    super().__init__()
    self.texts=df['Message'].values
    self.labels=df['Numeric_label'].values

  def __len__(self):
    return len(self.texts)

  def __getitem__(self,idx):
    text=self.texts[idx]
    label=self.labels[idx]

    word2vec= create_word2vec_for_setence(text,4)
    label=torch.tensor(label,dtype=torch.float32)

    return word2vec,label



NameError: ignored

In [ ]:
train_dataset,test_dataset=train_test_split(df, test_size=0.2, random_state=42)

train_my_dataset=MyDataset(train_dataset)
test_my_dataset=MyDataset(test_dataset)
train_dataloader=DataLoader(train_my_dataset,batch_size=32,shuffle=True)
test_dataloader=DataLoader(test_my_dataset,batch_size=32,shuffle=False)

In [ ]:
#update input_dim and hidden_dim
#why requires_grad=False
class MyModel(nn.Module):
  def __init__(self,input_dim=25,hidden_dim=10):
    super().__init__()
    self.input_dim=input_dim
    self.hidden_dim=hidden_dim
    self.linear_layer1=nn.Linear(self.input_dim,self.hidden_dim)
    self.linear_layer2=nn.Linear(self.hidden_dim,self.hidden_dim)
    self.final_layer=nn.Linear(self.hidden_dim,1)
    self.sigmoid=nn.Sigmoid()

  def forward(self,x):
    print(x.shape)

    x=torch.mean(x,1)
    print(x.shape)
    x.requires_grad=True

    #x=x.view(batch_size,-1)
    #x=torch.conacte()
    x=self.linear_layer1(x)
    x=self.sigmoid(x)
    x=self.linear_layer2(x)
    x=self.sigmoid(x)
    x=self.final_layer(x)
    #print('requires_grad',x.requires_grad)
    x=self.sigmoid(x)


    return x




In [ ]:
model=MyModel()

In [ ]:
criterion=nn.BCELoss(reduction='mean')

NameError: ignored

In [ ]:
optimizer=torch.optim.Adam(model.parameters(),lr=0.0001)

NameError: ignored

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:

for epoch in range(10):
  overall_accuracy=0
  for x,y in train_dataloader:
    predicted_y=model(x)

    batch_size=x.shape[0]
    y=y.view(batch_size,1)

    loss=criterion(predicted_y,y)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()
    y_true=y.detach().numpy()

    y_pred=predicted_y.detach().numpy() >0.5



    accuracy= accuracy_score(y_true,y_pred)

    overall_accuracy +=accuracy*batch_size


  print(f'Epoch: {epoch} --> Accuracy {overall_accuracy/len(train_my_dataset)}')



Epoch: 0 --> Accuracy 0.12242424242424242
Epoch: 1 --> Accuracy 0.12242424242424242
Epoch: 2 --> Accuracy 0.12242424242424242
Epoch: 3 --> Accuracy 0.45406060606060605
Epoch: 4 --> Accuracy 0.8775757575757576
Epoch: 5 --> Accuracy 0.8775757575757576
Epoch: 6 --> Accuracy 0.8775757575757576
Epoch: 7 --> Accuracy 0.8775757575757576
Epoch: 8 --> Accuracy 0.8775757575757576
Epoch: 9 --> Accuracy 0.8775757575757576


This week::
#1. root word for bangla dictionary(Matters)
#2. Bangla root word word2vec(pretrained)
#3. Suffix add based on rules->bangal->banglai jai->jaccilam khai->khaccilam (bangla stemmer papers)
